In [1]:
import pandas as pd
import glob
import os
import gutenberg
from gutenberg.cleanup import strip_headers
import sqlite3 as sql
import requests
import urllib
import json
from collections import Counter
from ast import literal_eval

In [2]:
conn = sql.connect('pg-pr-eng-v4-dbpedia.db')

In [3]:
df = pd.read_sql_query('select * from meta', conn)

In [4]:
shakes = pd.read_sql_query('select text, text_cleaned from text where id = 100', conn)

In [5]:
text = shakes.text_cleaned[0]

In [4]:
df

,level_0,index,id,_repo,gitenberg metadata version,alternative_title,contributor,covers,creator,description,...,hasPR,isEng,filename,plaintext,gutFilename,subjects,releaseDate,literaryGenres,info,wikiSubjects
0,0.0,0,100,The-Complete-Works-of-William-Shakespeare_100,0.1.0,None,None,None,"{'author': {'agent_name': 'Shakespeare, Willia...",None,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/The-Complete-...,http://www.gutenberg.org/ebooks/100.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/0/10...,['English drama -- Early modern and Elizabetha...,None,None,None,None
1,1.0,1,10002,The-House-on-the-Borderland_10002,0.0.1,None,None,None,"{'author': {'agent_name': 'Hodgson, William Ho...",The House on the Borderland (1908) is a supern...,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/The-House-on-...,http://www.gutenberg.org/ebooks/10002.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/0/0/...,['Science fiction'],None,None,None,None
2,2.0,2,10007,Carmilla_10007,0.0.1,None,None,None,"{'author': {'agent_name': 'Le Fanu, Joseph She...",Carmilla is a Gothic novella by Joseph Sherida...,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/Carmilla_1000...,http://www.gutenberg.org/ebooks/10007.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/0/0/...,"['Vampires -- Fiction', 'Young women -- Fiction']",None,None,None,None
3,3.0,3,10021,Tenterhooks_10021,0.0.1,None,None,None,"{'author': {'agent_name': 'Leverson, Ada', 'al...",None,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/Tenterhooks_1...,http://www.gutenberg.org/files/10021/10021.zip,/run/media/jon/SAMSUNG/gutenberg-mirror/1/0/0/...,"['Man-woman relationships -- Fiction', 'London...",None,None,None,None
4,4.0,4,10029,The-Hunt-Ball-Mystery_10029,0.0.1,None,None,None,"{'author': {'agent_name': 'Magnay, William, Si...",Cover -- Contents.,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/The-Hunt-Ball...,http://www.gutenberg.org/ebooks/10029.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/0/0/...,['Detective and mystery stories'],None,None,None,None
5,5.0,5,10039,The-Works-of-Aphra-Behn-Volume-III_10039,0.0.1,None,None,None,"{'author': {'agent_name': 'Behn, Aphra', 'alia...",None,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/The-Works-of-...,None,/run/media/jon/SAMSUNG/gutenberg-mirror/1/0/0/...,['English drama -- 17th century'],None,None,None,None
6,6.0,6,10046,Salute-to-Adventurers_10046,0.0.1,None,None,None,"{'author': {'agent_name': 'Buchan, John', 'ali...",In John Buchan's thrill-a-minute novel Salute ...,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/Salute-to-Adv...,http://www.gutenberg.org/files/10046/10046.zip,/run/media/jon/SAMSUNG/gutenberg-mirror/1/0/0/...,"['Virginia -- History -- Colonial period, ca. ...",None,None,None,None
7,7.0,7,10049,Old-Lady-MaryA-Story-of-the-Seen-and-the-Unsee...,0.0.1,None,None,None,"{'author': {'agent_name': 'Oliphant, Mrs. (Mar...",None,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/Old-Lady-Mary...,http://www.gutenberg.org/files/10049/10049.zip,/run/media/jon/SAMSUNG/gutenberg-mirror/1/0/0/...,"['Inheritance and succession -- Fiction', 'Dea...",None,None,None,None
8,8.0,8,10050,A-Little-Pilgrim--Stories-of-the-Seen-and-the-...,0.0.1,None,None,None,"{'author': {'agent_name': 'Oliphant, Mrs. (Mar...",None,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/A-Little-Pilg...,http://www.gutenberg.org/ebooks/10050.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/0/0/...,['Death -- Fiction'],None,None,None,None
9,9.0,9,10051,The-Little-Pilgrim--Further-Experiences.Storie...,0.0.1,None,None,None,"{'author': {'agent_name': 'Oliphant, Mrs. (Mar...",None,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/The-Little-Pi...,http://www.gutenberg.org/ebooks/10051.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/0/0/...,['Death -- Fiction'],None,None,None,None


In [5]:
df = pd.read_sql_query('select * from meta where author = "Dickens, Charles"', conn)

In [6]:
df

,level_0,index,id,_repo,gitenberg metadata version,alternative_title,contributor,covers,creator,description,...,hasPR,isEng,filename,plaintext,gutFilename,subjects,releaseDate,literaryGenres,info,wikiSubjects
0,596.0,596,1289,Three-Ghost-Stories_1289,0.0.1,None,None,None,"{'author': {'agent_name': 'Dickens, Charles', ...",Though best known for his heartwarming holiday...,...,1.0,1.0,None,None,None,"['English fiction', 'Ghost stories']",None,None,None,None
1,651.0,651,1392,The-Seven-Poor-Travellers_1392,0.2.0,None,None,"[{'cover_type': 'generated', 'image_path': 'co...","{'author': {'agent_name': 'Dickens, Charles', ...",In the old city of Rochester -- The story of R...,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/The-Seven-Poo...,http://www.gutenberg.org/ebooks/1392.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/3/9/...,"['Christmas stories, English', 'Fiction']",None,None,None,None
2,653.0,653,1394,The-Holly-Tree_1394,0.2.0,None,None,"[{'cover_type': 'generated', 'image_path': 'co...","{'author': {'agent_name': 'Dickens, Charles', ...",No author is more associated with Christmas-ti...,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/The-Holly-Tre...,http://www.gutenberg.org/ebooks/1394.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/3/9/...,"['English fiction', 'Christmas stories']",None,None,None,None
3,655.0,655,1406,The-Perils-of-Certain-English-Prisoners_1406,0.2.0,None,None,"[{'cover_type': 'generated', 'image_path': 'co...","{'author': {'agent_name': 'Dickens, Charles', ...",None,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/The-Perils-of...,http://www.gutenberg.org/ebooks/1406.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/4/0/...,['English fiction -- 19th century'],None,None,None,None
4,656.0,656,1407,A-Message-from-the-Sea_1407,0.2.0,None,None,"[{'cover_type': 'generated', 'image_path': 'co...","{'author': {'agent_name': 'Dickens, Charles', ...",One of a series of episodic tales that Charles...,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/A-Message-fro...,http://www.gutenberg.org/ebooks/1407.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/4/0/...,"['Sea stories', 'English fiction -- 19th centu...",None,None,None,None
5,659.0,659,1413,Tom-Tiddler-s-Ground_1413,0.2.0,None,None,"[{'cover_type': 'generated', 'image_path': 'co...","{'author': {'agent_name': 'Dickens, Charles', ...","""And why Tom Tiddler's ground?"" said the Trave...",...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/Tom-Tiddler-s...,http://www.gutenberg.org/ebooks/1413.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/4/1/...,['English fiction'],None,None,None,None
6,660.0,660,1414,Somebody-s-Luggage_1414,0.2.0,None,None,"[{'cover_type': 'generated', 'image_path': 'co...","{'author': {'agent_name': 'Dickens, Charles', ...","""My family-name is Blue-Beard! I'm going to op...",...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/Somebody-s-Lu...,http://www.gutenberg.org/ebooks/1414.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/4/1/...,['England -- Social life and customs -- 19th c...,None,None,None,None
7,661.0,661,1415,Doctor-Marigold_1415,0.2.0,None,None,"[{'cover_type': 'generated', 'image_path': 'co...","{'author': {'agent_name': 'Dickens, Charles', ...","Craving some feel-good fiction? Curl up with ""...",...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/Doctor-Marigo...,http://www.gutenberg.org/ebooks/1415.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/4/1/...,"['Adoptive parents -- Fiction', 'Deaf children...",None,None,None,None
8,662.0,662,1416,Mrs.-Lirriper-s-Lodgings_1416,0.2.0,None,None,"[{'cover_type': 'generated', 'image_path': 'co...","{'author': {'agent_name': 'Dickens, Charles', ...",Charles Dickens' absorbing tale of people thro...,...,1.0,1.0,/run/media/jon/SAMSUNG/gitenberg/Mrs.-Lirriper...,http://www.gutenberg.org/ebooks/1416.txt.utf-8,/run/media/jon/SAMSUNG/gutenberg-mirror/1/4/1/...,"['Widows -- Fiction', 'Boardinghouses -- Ficti...",None,None,None,None
9,664.0,664,1419,Mugby-Junction_1419,0.2.0,None,None,"[{'cover_type': 'ge